In [1]:
import pandas as pd
import numpy as np
import os
import glob
import datetime


In [2]:
def remove_decimals(df, column_name):
    """
    Removes Decimal points from columns in data frames.
    """
    try:
        df[column_name] = df[column_name].astype(str)
        m = df[column_name].str.contains('\d+')
        df.loc[m, column_name] = df.loc[m, column_name].apply(lambda x: x.split('.')[0])
        df.loc[m, column_name] = df.loc[m, column_name].astype('int64')
    except ValueError:
        pass

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.expand_frame_repr', True)

In [4]:
df_fi = pd.read_excel(r'C:\Users\shivani.kumari\Downloads\FI_DATA.xlsx',dtype={'BUKRS': str,'BELNR':'str','Document_type':'str','Company code':'str',"store_or_dc":'str','ZBD1T':'Int64','suppl_no':'str','LIFNR':'str','REDAT':'str','GEBRF':'str','GSMWB':'str','WEARS': 'str','GSMWF':'str','WENUM':'str','RGDAT':str,'ABGST':'str','VORGN':'str','GJAHR':'str','DEBNOTNO':'str'})

In [5]:
df_ic_mmsic = pd.read_excel(r'C:\Users\shivani.kumari\Downloads\MMSIC_IC.xlsx',dtype={'BELNR':'str','AUFNR':'str','WEDAT':'str','ZBD1T':'Int64','LIFNR':'str','REDAT':'str','GEBRF':'str','GSMWB':'str','WEARS': 'str','GSMWF':'str','WENUM':'str','RGDAT':'str','ABGST':'str','VORGN':'str','GJAHR':'str','VORGN':'str'})

df_ic_sis =  pd.read_excel(r'C:\Users\shivani.kumari\Downloads\SIS_IC.xlsx',dtype={'BELNR':'str','AUFNR':'str','WEDAT':'str','ZBD1T':'Int64','LIFNR':'str','REDAT':'str','GEBRF':'str','GSMWB':'str','WEARS': 'str','GSMWF':'str','WENUM':'str','RGDAT':'str','ABGST':'str','VORGN':'str','GJAHR':'str','VORGN':'str'})

df_ic = pd.concat([df_ic_mmsic,df_ic_sis],axis = 0, ignore_index = True )

In [6]:
sftp_df = pd.read_csv(r'C:\Users\shivani.kumari\Downloads\miag.35.304356.20240930.671.csv',dtype = {"Store":str,"Supplier number (MIAG)":str,"Remittance advice number":str,"Supplier number (Sales Line)":str,"Document number":str})

In [7]:
    positive_l6_rows = df_fi[(df_fi['Document_type'] == 'L6') & (
        (df_fi['Amount_in_document_currency'] >= 0))].index

    # Remove the selected rows
    df_fi.drop(positive_l6_rows, inplace=True)

    # Doc_type l6 sum groupes with doc number
    l6_sum = df_fi[df_fi['Document_type'] == 'L6'].groupby(
        ['BELNR', 'suppl_no'])['Amount_in_document_currency'].sum().reset_index()
    # zipped
    belnr_to_sum = dict(
        zip(list(zip(l6_sum['BELNR'], l6_sum['suppl_no'])), l6_sum['Amount_in_document_currency']))
    # giving all the l6 doc types their summed values
    df_fi['Amount_in_document_currency'] = df_fi.apply(lambda row: belnr_to_sum.get(
        (row['BELNR'], row['suppl_no']), row['Amount_in_document_currency']), axis=1)

    # keeping the second row of l6 doc type for each doc number
    l6_df_fi = df_fi[df_fi['Document_type'] == 'L6'].groupby(
        ['BELNR', 'suppl_no']).nth(1)

    # non l6 docs with shkzg == "S"
    non_l6_s_df_fi = df_fi[(df_fi['Document_type'] != 'L6') & (df_fi['shkzg'] == 'S')]
    non_l6_s_df_fi = non_l6_s_df_fi.sort_values(
        by=['BELNR', 'Amount_in_document_currency'], ascending=[True, True])
    non_l6_s_df_fi = non_l6_s_df_fi.drop_duplicates(
        subset=['BELNR', 'suppl_no', 'Amount_in_document_currency'], keep='first')

    # non l6 docs with shkzg == "H"
    non_l6_h_df_fi = df_fi[(df_fi['Document_type'] != 'L6') & (df_fi['shkzg'] == 'H')]
    non_l6_h_df_fi = non_l6_h_df_fi.drop_duplicates(
        subset=['BELNR', 'suppl_no', 'Amount_in_document_currency'], keep='first')

    # add shkzg = "H" and shkzg = "S" rows
    non_l6_df_fi = pd.concat([non_l6_s_df_fi, non_l6_h_df_fi], ignore_index=True)

    # keeping only rows whose currency is greater then 0
    positive_non_l6_rows = non_l6_df_fi[(
            non_l6_df_fi['Amount_in_document_currency'] >= 0)].index
    non_l6_df_fi.drop(positive_non_l6_rows, inplace=True)

    # concatinating
    df_fi = pd.concat([l6_df_fi, non_l6_df_fi], ignore_index=True)

In [8]:
# Count non-null values in the entire DataFrame, filtered by the specified column
df_ic['non_null_count'] = df_ic[df_ic['RENR'].notnull()].notnull().sum(axis=1)

# Select the row with the maximum non-null values in the entire row
most_complete_row = df_ic.loc[df_ic['non_null_count'].idxmax()]

# Drop the count column before displaying
most_complete_row = most_complete_row.drop('non_null_count')

print(most_complete_row)

LIFNR                  15545
BELNR              810187451
RENR        EI22024000000007
REDAT               20240207
LFSNR       00000000000ERS61
GEBRF              336019.83
GSMWB                      0
GSMWF                3326.93
WAERS                    TRY
WENUM                 290829
RGDAT               20240214
ABGST                   1204
AUFNR               45320377
VORGN         20240000064855
GJAHR                   2024
WEDAT               20240208
DEBNOTNO          24131977.0
Name: 3340, dtype: object


In [9]:
df_ic.shape

(28148, 18)

In [10]:
df_fi.shape

(3430, 35)

In [11]:
doc_to_type = dict(zip(df_ic['BELNR'], df_ic['RENR']))

    # Fill missing values in FI_df['XBLNR'] using the mapping and FI_df['BELNR']
df_fi['XBLNR'] = df_fi['XBLNR'].fillna(df_fi['BELNR'].map(doc_to_type))

In [12]:
doc_to_type = dict(zip(df_fi['XBLNR'], df_fi['BELNR']))

df_ic['BELNR'] = df_ic['BELNR'].str.strip().replace('', np.nan)
    # Fill missing values in CI_df['BELNR'] using the mapping and CI_df['RENR']
df_ic['BELNR'] = df_ic['BELNR'].fillna(df_ic['RENR'].map(doc_to_type))

standardise the SFTP data

In [13]:
    '''sftp_df['Supplier number (Sales Line)'] = sftp_df['Supplier number (Sales Line)'].astype(
        str)
    sftp_df["Supplier number (Sales Line)"] = sftp_df[
        "Supplier number (Sales Line)"
    ].str.replace("^10000", "", regex=True)
    sftp_df['Supplier number (Sales Line)'] = sftp_df['Supplier number (Sales Line)'].astype('int64')'''

'sftp_df[\'Supplier number (Sales Line)\'] = sftp_df[\'Supplier number (Sales Line)\'].astype(\n    str)\nsftp_df["Supplier number (Sales Line)"] = sftp_df[\n    "Supplier number (Sales Line)"\n].str.replace("^10000", "", regex=True)\nsftp_df[\'Supplier number (Sales Line)\'] = sftp_df[\'Supplier number (Sales Line)\'].astype(\'int64\')'

check the lengths and dtypes

In [14]:
'''measurer = np.vectorize(len)
res1 = measurer(df_fi.values.astype(str)).max(axis=0)
res1 = dict(zip(df_fi, measurer(df_fi.values.astype(str)).max(axis=0)))
print(res1)
print(df_fi.dtypes)'''

'measurer = np.vectorize(len)\nres1 = measurer(df_fi.values.astype(str)).max(axis=0)\nres1 = dict(zip(df_fi, measurer(df_fi.values.astype(str)).max(axis=0)))\nprint(res1)\nprint(df_fi.dtypes)'

In [15]:
'''measurer = np.vectorize(len)
res2 = measurer(df_ic.values.astype(str)).max(axis=0)
res2 = dict(zip(df_ic, measurer(df_ic.values.astype(str)).max(axis=0)))
print(res2)
print(df_ic.dtypes)'''

'measurer = np.vectorize(len)\nres2 = measurer(df_ic.values.astype(str)).max(axis=0)\nres2 = dict(zip(df_ic, measurer(df_ic.values.astype(str)).max(axis=0)))\nprint(res2)\nprint(df_ic.dtypes)'

In [16]:
'''measurer = np.vectorize(len)
res3 = measurer(df_sftp.values.astype(str)).max(axis=0)
print(res3)
print(df_sftp.dtypes)'''

'measurer = np.vectorize(len)\nres3 = measurer(df_sftp.values.astype(str)).max(axis=0)\nprint(res3)\nprint(df_sftp.dtypes)'

standarized the data

In [17]:
#preprocess ic/FI/sftp document numbers. make the length 10 or above

df_ic['LIFNR'] = df_ic['LIFNR'].apply(
    lambda x: '10000' + str(x) if not pd.isna(x) and len(str(x)) < 10 else str(x) if not pd.isna(x) else '0' * 10
)

df_fi['suppl_no'] = df_fi['suppl_no'].apply(
    lambda x: '10000' + str(x) if not pd.isna(x) and len(str(x)) < 10 else str(x) if not pd.isna(x) else '0' * 10
)

df_ic['BELNR'] = df_ic['BELNR'].apply(
     lambda x: '0' * (10 - len(str(x))) + str(x) if not pd.isna(x) else '0' * 10)

df_fi['BELNR'] = df_fi['BELNR'].apply(
     lambda x: '0' * (10 - len(str(x))) + str(x) if not pd.isna(x) else '0' * 10)

sftp_df['Document number'] = sftp_df['Document number'].apply(
     lambda x: '0' * (10 - len(str(x))) + str(x) if not pd.isna(x) else '0' * 10)
     


In [18]:
df_fi

MANDT Document_type document_type_desc GJAHR BUKRS  GSBER       PRCTR  \
0       100            L6        AP good-BAS  2024  3142    NaN  31421037.0   
1       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
2       100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
3       100            L6        AP good-BAS  2024  3142    NaN  31423382.0   
4       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
5       100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
6       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
7       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
8       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
9       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
10      100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
11      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
12      100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
13      100            L6        AP good-BAS  2024  3142    NaN  31421033.0   
14      100            L6        AP good-BAS  2024  3142    NaN  31423386.0   
15      100            L6        AP good-BAS  2024  3142    NaN  31421043.0   
16      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
17      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
18      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
19      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
20      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
21      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
22      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
23      100            L6        AP good-BAS  2024  3142    NaN         NaN   
24      100            L6        AP good-BAS  2024  3142    NaN         NaN   
25      100            L6        AP good-BAS  2024  3142    NaN         NaN   
26      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
27      100            L6        AP good-BAS  2024  3142    NaN         NaN   
28      100            L6        AP good-BAS  2024  3142    NaN         NaN   
29      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
30      100            L6        AP good-BAS  2024  3142    NaN         NaN   
31      100            L6        AP good-BAS  2024  3142    NaN         NaN   
32      100            L6        AP good-BAS  2024  3142    NaN         NaN   
33      100            L6        AP good-BAS  2024  3142    NaN         NaN   
34      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
35      100            L6        AP good-BAS  2024  3142    NaN         NaN   
36      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
37      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
38      100            L6        AP good-BAS  2024  3142    NaN  31423378.0   
39      100            L6        AP good-BAS  2024  3142    NaN  31421044.0   
40      100            L6        AP good-BAS  2024  3142    NaN  31423377.0   
41      100            L6        AP good-BAS  2024  3142    NaN  31421053.0   
42      100            L6        AP good-BAS  2024  3142    NaN  31423389.0   
43      100            L6        AP good-BAS  2024  3142    NaN  31421037.0   
44      100            L6        AP good-BAS  2024  3142    NaN  31421034.0   
45      100            L6        AP good-BAS  2024  3142    NaN  31423379.0   
46      100            L6        AP good-BAS  2024  3142    NaN  31421053.0   
47      100            L6        AP good-BAS  2024  3142    NaN  31423385.0   
48      100            L6        AP good-BAS  2024  3142    NaN  31421023.0   
49      100            L6        AP good-BAS  2024  3142

In [19]:
def transform_number(num):
    if pd.isna(num):  # Check for NaN values
        return np.nan   # Return NaN if the input is NaN
    num_str = str(num)  # Convert to string

    # If the number already starts with '10000', return it as is
    if num_str.startswith('10000'):
        return num_str
    
    stripped_num = num_str.lstrip('0')  # Remove leading zeros
    final_num = stripped_num.zfill(6)  # Ensure it has at least 6 digits
    return '1000' + final_num  # Prepend '10000'

# Apply the function to the relevant column

df_ic['LIFNR'] = df_ic['LIFNR'].apply(transform_number)

In [20]:
df_fi

MANDT Document_type document_type_desc GJAHR BUKRS  GSBER       PRCTR  \
0       100            L6        AP good-BAS  2024  3142    NaN  31421037.0   
1       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
2       100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
3       100            L6        AP good-BAS  2024  3142    NaN  31423382.0   
4       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
5       100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
6       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
7       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
8       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
9       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
10      100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
11      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
12      100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
13      100            L6        AP good-BAS  2024  3142    NaN  31421033.0   
14      100            L6        AP good-BAS  2024  3142    NaN  31423386.0   
15      100            L6        AP good-BAS  2024  3142    NaN  31421043.0   
16      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
17      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
18      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
19      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
20      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
21      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
22      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
23      100            L6        AP good-BAS  2024  3142    NaN         NaN   
24      100            L6        AP good-BAS  2024  3142    NaN         NaN   
25      100            L6        AP good-BAS  2024  3142    NaN         NaN   
26      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
27      100            L6        AP good-BAS  2024  3142    NaN         NaN   
28      100            L6        AP good-BAS  2024  3142    NaN         NaN   
29      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
30      100            L6        AP good-BAS  2024  3142    NaN         NaN   
31      100            L6        AP good-BAS  2024  3142    NaN         NaN   
32      100            L6        AP good-BAS  2024  3142    NaN         NaN   
33      100            L6        AP good-BAS  2024  3142    NaN         NaN   
34      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
35      100            L6        AP good-BAS  2024  3142    NaN         NaN   
36      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
37      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
38      100            L6        AP good-BAS  2024  3142    NaN  31423378.0   
39      100            L6        AP good-BAS  2024  3142    NaN  31421044.0   
40      100            L6        AP good-BAS  2024  3142    NaN  31423377.0   
41      100            L6        AP good-BAS  2024  3142    NaN  31421053.0   
42      100            L6        AP good-BAS  2024  3142    NaN  31423389.0   
43      100            L6        AP good-BAS  2024  3142    NaN  31421037.0   
44      100            L6        AP good-BAS  2024  3142    NaN  31421034.0   
45      100            L6        AP good-BAS  2024  3142    NaN  31423379.0   
46      100            L6        AP good-BAS  2024  3142    NaN  31421053.0   
47      100            L6        AP good-BAS  2024  3142    NaN  31423385.0   
48      100            L6        AP good-BAS  2024  3142    NaN  31421023.0   
49      100            L6        AP good-BAS  2024  3142

In [21]:
#PREPROCESS DATE TIME. make all %d.%m.%Y

for col in df_ic.select_dtypes(include=['datetime64[ns]', 'datetime']):
    df_ic[col] = df_ic[col].dt.strftime('%d.%m.%Y')

for col in df_fi.select_dtypes(include=['datetime64[ns]', 'datetime']):
    df_fi[col] = df_fi[col].dt.strftime('%d.%m.%Y')
    

In [22]:
#convert string types into %d.%m.%Y. IC frame
for col in df_ic.select_dtypes(include=['object']):
    try:
        # Convert to datetime, assuming the format is 'YYYYMMDD'
        df_ic[col] = pd.to_datetime(df_ic[col], format='%Y%m%d')
        # Optionally, you can convert it to DMY format
        df_ic[col] = df_ic[col].dt.strftime('%d.%m.%Y')
    except ValueError:
        # If conversion fails, just continue
        pass

In [23]:
#convert string types into %d.%m.%Y. IC frame
for col in df_fi.select_dtypes(include=['object']):
    try:
        # Convert to datetime, assuming the format is 'YYYYMMDD'
        df_fi[col] = pd.to_datetime(df_fi[col], format='%Y%m%d')
        # Optionally, you can convert it to DMY format
        df_fi[col] = df_fi[col].dt.strftime('%d.%m.%Y')
    except ValueError:
        # If conversion fails, just continue
        pass

In [24]:
df_ic

LIFNR       BELNR              RENR       REDAT             LFSNR  \
0      1000020870  0810160734  A062024000000046  03.02.2024          13622860   
1      1000020870  0810160569  A062024000000049  05.02.2024  5222024000000276   
2      1000020870  0810160735  A062024000000047  03.02.2024          13625870   
3      1000020870  0810170024  A642024000000148  02.02.2024  6702024000000398   
4      1000020870  0810170025  A642024000000149  02.02.2024  6702024000000399   
5      1000020870  0810158452  A642024000000150  02.02.2024  6702024000000401   
6      1000020870  0810160734  A062024000000046  03.02.2024          13622860   
7      1000020870  0810160569  A062024000000049  05.02.2024  5222024000000276   
8      1000020870  0810160735  A062024000000047  03.02.2024          13625870   
9      1000020870  0810163617  A072024000000018  02.02.2024  5302024000000311   
10     1000020870  0810174184  A252024000000130  02.02.2024     2024000000430   
11     1000020870  0810175130  A252024000000131  02.02.2024     2024000000431   
12     1000020870  0810163447  A252024000000132  02.02.2024     2024000000432   
13     1000020870  0810162862  A582024000000022  03.02.2024  6202024000000321   
14     1000020870  0810162782  A262024000000371  05.02.2024  6002024000004412   
15     1000020870  0000000000  A072024000000015  01.02.2024  5312024000000292   
16     1000020870  0000000000  A072024000000016  01.02.2024  5312024000000297   
17     1000020870  0000000000  A082024000000533  01.02.2024  5302024000002305   
18     1000020870  0000000000  A082024000000534  01.02.2024  5302024000002310   
19     1000020870  0000000000  A082024000000535  01.02.2024  5302024000002336   
20     1000020870  0000000000  A202024000000099  01.02.2024  5742024000000705   
21     1000020870  0810161159  A532024000000812  02.02.2024  6102024000003313   
22     1000020870  0810161269  A082024000000542  02.02.2024   530202400002402   
23     1000020870  0810161316  A262024000000373  05.02.2024  6002024000004413   
24     1000020870  0810161409  A132024000000286  02.02.2024              2249   
25     1000020870  0810161414  A582024000000023  03.02.2024  6202024000000323   
26     1000020870  0810161446  A532024000000811  02.02.2024     6102024003323   
27     1000020870  0810161493  A592024000000055  02.02.2024  6302024000000435   
28     1000020870  0810161494  A592024000000056  02.02.2024  6302024000000433   
29     1000020870  0810161667  A532024000000816  02.02.2024  6102024000003385   
30     1000020870  0810161705  A082024000000543  02.02.2024  5302024000002404   
31     1000020870  0810161737  A262024000000372  05.02.2024  6002024000004405   
32     1000020870  0810161845  A532024000000815  02.02.2024  6102024000003328   
33     1000020870  0810161990  A252024000000133  02.02.2024     2024000000433   
34     1000020870  0810162021  A532024000000813  02.02.2024  6102024000003303   
35     1000020870  0810162123  A132024000000285  02.02.2024   550202400002224   
36     1000020870  0810162171  A072024000000017  02.02.2024  5312024000000310   
37     1000020870  0810162197  A532024000000814  02.02.2024  6102024000003365   
38     1000020870  0810162213  A132024000000287  02.02.2024   550202400002221   
39     1000020870  0810163617  A072024000000018  02.02.2024  5302024000000311   
40     1000020870  0810174184  A252024000000130  02.02.2024     2024000000430   
41     1000020870  0810175130  A252024000000131  02.02.2024     2024000000431   
42     1000020870  0810163447  A252024000000132  02.02.2024     2024000000432   
43     1000020870  0810162862  A582024000000022  03.02.2024  6202024000000321   
44     1000020870  0810162782  A262024000000371  05.02.2024  6002024000004412   
45     1000020870  0000000000  A072024000000015  01.02.2024  5312024000000292   
46     1000020870  0000000000  A072024000000016  01.02.2024  5312024000000297   
47     1000020870  0000000000  A082024000000533  01.02.2024  5302024000002305   
48     1000020870  0000000000  A08202400000

In [25]:
df_fi

MANDT Document_type document_type_desc GJAHR BUKRS  GSBER       PRCTR  \
0       100            L6        AP good-BAS  2024  3142    NaN  31421037.0   
1       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
2       100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
3       100            L6        AP good-BAS  2024  3142    NaN  31423382.0   
4       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
5       100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
6       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
7       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
8       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
9       100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
10      100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
11      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
12      100            L6        AP good-BAS  2024  3142    NaN  31426091.0   
13      100            L6        AP good-BAS  2024  3142    NaN  31421033.0   
14      100            L6        AP good-BAS  2024  3142    NaN  31423386.0   
15      100            L6        AP good-BAS  2024  3142    NaN  31421043.0   
16      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
17      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
18      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
19      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
20      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
21      100            L6        AP good-BAS  2024  3142    NaN  31426092.0   
22      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
23      100            L6        AP good-BAS  2024  3142    NaN         NaN   
24      100            L6        AP good-BAS  2024  3142    NaN         NaN   
25      100            L6        AP good-BAS  2024  3142    NaN         NaN   
26      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
27      100            L6        AP good-BAS  2024  3142    NaN         NaN   
28      100            L6        AP good-BAS  2024  3142    NaN         NaN   
29      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
30      100            L6        AP good-BAS  2024  3142    NaN         NaN   
31      100            L6        AP good-BAS  2024  3142    NaN         NaN   
32      100            L6        AP good-BAS  2024  3142    NaN         NaN   
33      100            L6        AP good-BAS  2024  3142    NaN         NaN   
34      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
35      100            L6        AP good-BAS  2024  3142    NaN         NaN   
36      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
37      100            L6        AP good-BAS  2024  3142    NaN  31420100.0   
38      100            L6        AP good-BAS  2024  3142    NaN  31423378.0   
39      100            L6        AP good-BAS  2024  3142    NaN  31421044.0   
40      100            L6        AP good-BAS  2024  3142    NaN  31423377.0   
41      100            L6        AP good-BAS  2024  3142    NaN  31421053.0   
42      100            L6        AP good-BAS  2024  3142    NaN  31423389.0   
43      100            L6        AP good-BAS  2024  3142    NaN  31421037.0   
44      100            L6        AP good-BAS  2024  3142    NaN  31421034.0   
45      100            L6        AP good-BAS  2024  3142    NaN  31423379.0   
46      100            L6        AP good-BAS  2024  3142    NaN  31421053.0   
47      100            L6        AP good-BAS  2024  3142    NaN  31423385.0   
48      100            L6        AP good-BAS  2024  3142    NaN  31421023.0   
49      100            L6        AP good-BAS  2024  3142

In [48]:
#merge FI and IC

merge_fi_ic = df_fi.merge(df_ic, left_on=[ "BELNR","XBLNR"], right_on=[
         "BELNR","RENR"], how="left")


In [49]:
df_fi.shape

(3430, 35)

In [50]:
'''print(merge_fi_ic.shape)
print(merge_fi_ic.columns)
print(merge_fi_ic.dtypes)'''

'print(merge_fi_ic.shape)\nprint(merge_fi_ic.columns)\nprint(merge_fi_ic.dtypes)'

In [51]:
merge_fi_ic

MANDT Document_type document_type_desc GJAHR_x BUKRS  GSBER  \
0        100            L6        AP good-BAS    2024  3142    NaN   
1        100            L6        AP good-BAS    2024  3142    NaN   
2        100            L6        AP good-BAS    2024  3142    NaN   
3        100            L6        AP good-BAS    2024  3142    NaN   
4        100            L6        AP good-BAS    2024  3142    NaN   
5        100            L6        AP good-BAS    2024  3142    NaN   
6        100            L6        AP good-BAS    2024  3142    NaN   
7        100            L6        AP good-BAS    2024  3142    NaN   
8        100            L6        AP good-BAS    2024  3142    NaN   
9        100            L6        AP good-BAS    2024  3142    NaN   
10       100            L6        AP good-BAS    2024  3142    NaN   
11       100            L6        AP good-BAS    2024  3142    NaN   
12       100            L6        AP good-BAS    2024  3142    NaN   
13       100            L6        AP good-BAS    2024  3142    NaN   
14       100            L6        AP good-BAS    2024  3142    NaN   
15       100            L6        AP good-BAS    2024  3142    NaN   
16       100            L6        AP good-BAS    2024  3142    NaN   
17       100            L6        AP good-BAS    2024  3142    NaN   
18       100            L6        AP good-BAS    2024  3142    NaN   
19       100            L6        AP good-BAS    2024  3142    NaN   
20       100            L6        AP good-BAS    2024  3142    NaN   
21       100            L6        AP good-BAS    2024  3142    NaN   
22       100            L6        AP good-BAS    2024  3142    NaN   
23       100            L6        AP good-BAS    2024  3142    NaN   
24       100            L6        AP good-BAS    2024  3142    NaN   
25       100            L6        AP good-BAS    2024  3142    NaN   
26       100            L6        AP good-BAS    2024  3142    NaN   
27       100            L6        AP good-BAS    2024  3142    NaN   
28       100            L6        AP good-BAS    2024  3142    NaN   
29       100            L6        AP good-BAS    2024  3142    NaN   
30       100            L6        AP good-BAS    2024  3142    NaN   
31       100            L6        AP good-BAS    2024  3142    NaN   
32       100            L6        AP good-BAS    2024  3142    NaN   
33       100            L6        AP good-BAS    2024  3142    NaN   
34       100            L6        AP good-BAS    2024  3142    NaN   
35       100            L6        AP good-BAS    2024  3142    NaN   
36       100            L6        AP good-BAS    2024  3142    NaN   
37       100            L6        AP good-BAS    2024  3142    NaN   
38       100            L6        AP good-BAS    2024  3142    NaN   
39       100            L6        AP good-BAS    2024  3142    NaN   
40       100            L6        AP good-BAS    2024  3142    NaN   
41       100            L6        AP good-BAS    2024  3142    NaN   
42       100            L6        AP good-BAS    2024  3142    NaN   
43       100            L6        AP good-BAS    2024  3142    NaN   
44       100            L6        AP good-BAS    2024  3142    NaN   
45       100            L6        AP good-BAS    2024  3142    NaN   
46       100            L6        AP good-BAS    2024  3142    NaN   
47       100            L6        AP good-BAS    2024  3142    NaN   
48       100            L6        AP good-BAS    2024  3142    NaN   
49       100            L6        AP good-BAS    2024  3142    NaN   
...      ...           ...                ...     ...   ...    ...   
22673    100            WN            Returns    2024  3142    NaN   
22674    100            WN            Returns    2024  3142    NaN   
22675    100            WN            Returns    2024  3142    NaN   
22676    100            WN            Returns    2024  3142    NaN   
22677    100            WN            Returns    2024  3142    NaN   
22678    100     

In [52]:
print(merge_fi_ic.shape)

(22723, 52)


In [53]:
#print(merge_fi_ic.dtypes)

In [54]:
#check lengths and dtyes of merged frame
measurer = np.vectorize(len)
res = measurer(merge_fi_ic.values.astype(str)).max(axis=0)
res = dict(zip(merge_fi_ic, measurer(merge_fi_ic.values.astype(str)).max(axis=0)))
print(res)

{'MANDT': np.int64(3), 'Document_type': np.int64(2), 'document_type_desc': np.int64(20), 'GJAHR_x': np.int64(4), 'BUKRS': np.int64(4), 'GSBER': np.int64(3), 'PRCTR': np.int64(10), 'store_or_dc': np.int64(4), 'KOSTL': np.int64(12), 'month_in_fin_year': np.int64(2), 'BELNR': np.int64(10), 'XBLNR': np.int64(16), 'AUGBL': np.int64(12), 'AUGDT': np.int64(10), 'ZFBDT': np.int64(10), 'ZBD1T': np.int64(4), 'ZBD2T': np.int64(3), 'NETDT': np.int64(3), 'BUZEI': np.int64(3), 'altkt': np.int64(10), 'hkont': np.int64(10), 'suppl_no': np.int64(10), 'BLDAT': np.int64(10), 'BUDAT': np.int64(10), 'CPUDT': np.int64(10), 'partition_date': np.int64(10), 'dana_ingestion_date': np.int64(10), 'shkzg': np.int64(1), 'Amount_in_local_currency': np.int64(11), 'Amount_in_document_currency': np.int64(19), 'Tax_in_local_currency': np.int64(3), 'Tax_in_document_currency': np.int64(3), 'WAERS_x': np.int64(3), 'Batch_Input_session_name': np.int64(12), 'sgtxt': np.int64(36), 'LIFNR': np.int64(10), 'RENR': np.int64(16), 

In [55]:
#merge_fi_ic['BELNR'] = merge_fi_ic['BELNR'].apply(
#        lambda x: '0' * (10 - len(str(x))) + str(x) if not pd.isna(x) else '0' * 10)

In [56]:
sftp_df

Contract area Supplier number (Sales Line) Supplier number (MIAG)  \
0               35                   1000010074             3500000073   
1               35                   1000010074             3500000073   
2               35                   1000010074             3500000073   
3               35                   1000010074             3500000073   
4               35                   1000010074             3500000073   
5               35                   1000010074             3500000073   
6               35                   1000010074             3500000073   
7               35                   1000010074             3500000073   
8               35                   1000010074             3500000073   
9               35                   1000010074             3500000073   
10              35                   1000010074             3500000073   
11              35                   1000010074             3500000073   
12              35                   1000010074             3500000073   
13              35                   1000010074             3500000073   
14              35                   1000015545             3500003274   
15              35                   1000015545             3500003274   
16              35                   1000015545             3500003274   
17              35                   1000015545             3500003274   
18              35                   1000015545             3500003274   
19              35                   1000015545             3500003274   
20              35                   1000015545             3500003274   
21              35                   1000015545             3500003274   
22              35                   1000015545             3500003274   
23              35                   1000015545             3500003274   
24              35                   1000015545             3500003274   
25              35                   1000015545             3500003274   
26              35                   1000015545             3500003274   
27              35                   1000015545             3500003274   
28              35                   1000015545             3500003274   
29              35                   1000020870             3500007956   
30              35                   1000020870             3500007956   
31              35                   1000020870             3500007956   
32              35                   1000020870             3500007956   
33              35                   1000020870             3500007956   
34              35                   1000020870             3500007956   
35              35                   1000020870             3500007956   
36              35                   1000020870             3500007956   
37              35                   1000020870             3500007956   
38              35                   1000020870             3500007956   
39              35                   1000020870             3500007956   
40              35                   1000020870             3500007956   
41              35                   1000020870             3500007956   
42              35                   1000020870             3500007956   
43              35                   1000020870             3500007956   
44              35                   1000020870             3500007956   
45              35                   1000020870             3500007956   
46              35                   1000020870             3500007956   
47              35                   1000020870             3500007956   
48              35                   1000020870             3500007956   
49              35                   1000020870             3500007956   
..             ...                          ...                    ...   
622             35                   1000020870             3500007956   
623             35                   1000020870             3500007956   
624      

net due date calculations

In [57]:
### sftp with merged, ic and fi data

merged_df = merge_fi_ic.merge(
        sftp_df, left_on=["BELNR", "XBLNR"],
        right_on=["Document number", "Invoice number"],
        how="left")

In [58]:
#merged_df.columns

In [59]:
merged_df

MANDT Document_type document_type_desc GJAHR_x BUKRS  GSBER  \
0        100            L6        AP good-BAS    2024  3142    NaN   
1        100            L6        AP good-BAS    2024  3142    NaN   
2        100            L6        AP good-BAS    2024  3142    NaN   
3        100            L6        AP good-BAS    2024  3142    NaN   
4        100            L6        AP good-BAS    2024  3142    NaN   
5        100            L6        AP good-BAS    2024  3142    NaN   
6        100            L6        AP good-BAS    2024  3142    NaN   
7        100            L6        AP good-BAS    2024  3142    NaN   
8        100            L6        AP good-BAS    2024  3142    NaN   
9        100            L6        AP good-BAS    2024  3142    NaN   
10       100            L6        AP good-BAS    2024  3142    NaN   
11       100            L6        AP good-BAS    2024  3142    NaN   
12       100            L6        AP good-BAS    2024  3142    NaN   
13       100            L6        AP good-BAS    2024  3142    NaN   
14       100            L6        AP good-BAS    2024  3142    NaN   
15       100            L6        AP good-BAS    2024  3142    NaN   
16       100            L6        AP good-BAS    2024  3142    NaN   
17       100            L6        AP good-BAS    2024  3142    NaN   
18       100            L6        AP good-BAS    2024  3142    NaN   
19       100            L6        AP good-BAS    2024  3142    NaN   
20       100            L6        AP good-BAS    2024  3142    NaN   
21       100            L6        AP good-BAS    2024  3142    NaN   
22       100            L6        AP good-BAS    2024  3142    NaN   
23       100            L6        AP good-BAS    2024  3142    NaN   
24       100            L6        AP good-BAS    2024  3142    NaN   
25       100            L6        AP good-BAS    2024  3142    NaN   
26       100            L6        AP good-BAS    2024  3142    NaN   
27       100            L6        AP good-BAS    2024  3142    NaN   
28       100            L6        AP good-BAS    2024  3142    NaN   
29       100            L6        AP good-BAS    2024  3142    NaN   
30       100            L6        AP good-BAS    2024  3142    NaN   
31       100            L6        AP good-BAS    2024  3142    NaN   
32       100            L6        AP good-BAS    2024  3142    NaN   
33       100            L6        AP good-BAS    2024  3142    NaN   
34       100            L6        AP good-BAS    2024  3142    NaN   
35       100            L6        AP good-BAS    2024  3142    NaN   
36       100            L6        AP good-BAS    2024  3142    NaN   
37       100            L6        AP good-BAS    2024  3142    NaN   
38       100            L6        AP good-BAS    2024  3142    NaN   
39       100            L6        AP good-BAS    2024  3142    NaN   
40       100            L6        AP good-BAS    2024  3142    NaN   
41       100            L6        AP good-BAS    2024  3142    NaN   
42       100            L6        AP good-BAS    2024  3142    NaN   
43       100            L6        AP good-BAS    2024  3142    NaN   
44       100            L6        AP good-BAS    2024  3142    NaN   
45       100            L6        AP good-BAS    2024  3142    NaN   
46       100            L6        AP good-BAS    2024  3142    NaN   
47       100            L6        AP good-BAS    2024  3142    NaN   
48       100            L6        AP good-BAS    2024  3142    NaN   
49       100            L6        AP good-BAS    2024  3142    NaN   
...      ...           ...                ...     ...   ...    ...   
22673    100            WN            Returns    2024  3142    NaN   
22674    100            WN            Returns    2024  3142    NaN   
22675    100            WN            Returns    2024  3142    NaN   
22676    100            WN            Returns    2024  3142    NaN   
22677    100            WN            Returns    2024  3142    NaN   
22678    100     

populate ARKTX

In [60]:
# Assuming merged_df is your DataFrame
merged_df["year"] = merged_df.BLDAT.apply(lambda x: x.split('.')[2] if isinstance(x, str) and len(x.split('.')) > 2 else None)
merged_df["month"] = merged_df.BLDAT.apply(lambda x: x.split('.')[1] if isinstance(x, str) and len(x.split('.')) > 1 else None)
merged_df["day"] = merged_df.BLDAT.apply(lambda x: x.split('.')[0] if isinstance(x, str) and len(x.split('.')) > 0 else None)


merged_df["ARKTX_temp"] = merged_df["Document_type"].astype(str) + "#" + merged_df["BELNR"] + "#" + merged_df[
        "year"] + merged_df["month"] + merged_df["day"] + "#" + merged_df["year"] + "#" + "00"+merged_df["BUZEI"].astype(str)

merged_df['ARKTX'] = merged_df['ARKTX'].str.strip().replace('', np.nan)

merged_df.loc[merged_df["ARKTX"] == 'NaN', 'ARKTX'] = merged_df["ARKTX_temp"]

net due calculations

In [61]:
merged_df['ZFBDT'] = pd.to_datetime(merged_df['ZFBDT'], format='%d.%m.%Y')

merged_df['ZBD1T'] = pd.to_numeric(merged_df['ZBD1T'], errors='coerce')
merged_df['ZBD2T'] = pd.to_numeric(merged_df['ZBD2T'], errors='coerce')

merged_df['NET_DUE_DATE'] = merged_df.apply(
    lambda row: row['ZFBDT'] + pd.Timedelta(days=row['ZBD1T'] if pd.notna(row['ZBD1T']) else (row['ZBD2T'] if pd.notna(row['ZBD2T']) else 0)),
    axis=1
)
merged_df['NET_DUE_DATE'] = merged_df['NET_DUE_DATE'].dt.strftime('%d.%m.%Y')

Invoice status assignment

In [62]:
for index, row in merged_df.iterrows():
        doc_number = row['BELNR']
        supplier_number = row['suppl_no']

        # Check if doc_number is present in sftp_df
        if (doc_number in sftp_df['Document number'].values) and (
                supplier_number in sftp_df['Supplier number (Sales Line)'].values):
            matching_row = sftp_df[(sftp_df['Document number'] == doc_number) & (
                        sftp_df['Supplier number (Sales Line)'] == supplier_number)]
            # Check if 'rem_number' is not empty in merged_df
            if not matching_row['Remittance advice number'].empty and matching_row['Remittance advice number'].values[
                0] != '':
                merged_df.at[index, 'INVOICE_STATUS'] = 'CLEARED'
                merged_df.at[index, 'AUGDT'] = matching_row['Value date'].iloc[0]
            else:
                # # Check if doc_number is present in FI data
                matching_row = df_ic[(df_ic['BELNR'] == str(doc_number)) & (
                            df_ic['LIFNR'].astype('int64') == np.int64(str(supplier_number)[1:]))]
                # print(supplier_number)

                if len(matching_row) == 0:
                    merged_df.at[index, 'INVOICE_STATUS'] = 'INVOICE APPROVAL COMPLETED'

else:
            # Check if doc_number is present in q1-df
            # print(supplier_number)
            matching_row = df_ic[(df_ic['BELNR'] == str(doc_number)) & (
                        df_ic['LIFNR'].astype('int64') == np.int64(str(supplier_number)[1:]))]

            if len(matching_row) == 0:
                merged_df.at[index, 'INVOICE_STATUS'] = 'INVOICE APPROVAL COMPLETED'

In [63]:
merged_df

MANDT Document_type document_type_desc GJAHR_x BUKRS  GSBER  \
0        100            L6        AP good-BAS    2024  3142    NaN   
1        100            L6        AP good-BAS    2024  3142    NaN   
2        100            L6        AP good-BAS    2024  3142    NaN   
3        100            L6        AP good-BAS    2024  3142    NaN   
4        100            L6        AP good-BAS    2024  3142    NaN   
5        100            L6        AP good-BAS    2024  3142    NaN   
6        100            L6        AP good-BAS    2024  3142    NaN   
7        100            L6        AP good-BAS    2024  3142    NaN   
8        100            L6        AP good-BAS    2024  3142    NaN   
9        100            L6        AP good-BAS    2024  3142    NaN   
10       100            L6        AP good-BAS    2024  3142    NaN   
11       100            L6        AP good-BAS    2024  3142    NaN   
12       100            L6        AP good-BAS    2024  3142    NaN   
13       100            L6        AP good-BAS    2024  3142    NaN   
14       100            L6        AP good-BAS    2024  3142    NaN   
15       100            L6        AP good-BAS    2024  3142    NaN   
16       100            L6        AP good-BAS    2024  3142    NaN   
17       100            L6        AP good-BAS    2024  3142    NaN   
18       100            L6        AP good-BAS    2024  3142    NaN   
19       100            L6        AP good-BAS    2024  3142    NaN   
20       100            L6        AP good-BAS    2024  3142    NaN   
21       100            L6        AP good-BAS    2024  3142    NaN   
22       100            L6        AP good-BAS    2024  3142    NaN   
23       100            L6        AP good-BAS    2024  3142    NaN   
24       100            L6        AP good-BAS    2024  3142    NaN   
25       100            L6        AP good-BAS    2024  3142    NaN   
26       100            L6        AP good-BAS    2024  3142    NaN   
27       100            L6        AP good-BAS    2024  3142    NaN   
28       100            L6        AP good-BAS    2024  3142    NaN   
29       100            L6        AP good-BAS    2024  3142    NaN   
30       100            L6        AP good-BAS    2024  3142    NaN   
31       100            L6        AP good-BAS    2024  3142    NaN   
32       100            L6        AP good-BAS    2024  3142    NaN   
33       100            L6        AP good-BAS    2024  3142    NaN   
34       100            L6        AP good-BAS    2024  3142    NaN   
35       100            L6        AP good-BAS    2024  3142    NaN   
36       100            L6        AP good-BAS    2024  3142    NaN   
37       100            L6        AP good-BAS    2024  3142    NaN   
38       100            L6        AP good-BAS    2024  3142    NaN   
39       100            L6        AP good-BAS    2024  3142    NaN   
40       100            L6        AP good-BAS    2024  3142    NaN   
41       100            L6        AP good-BAS    2024  3142    NaN   
42       100            L6        AP good-BAS    2024  3142    NaN   
43       100            L6        AP good-BAS    2024  3142    NaN   
44       100            L6        AP good-BAS    2024  3142    NaN   
45       100            L6        AP good-BAS    2024  3142    NaN   
46       100            L6        AP good-BAS    2024  3142    NaN   
47       100            L6        AP good-BAS    2024  3142    NaN   
48       100            L6        AP good-BAS    2024  3142    NaN   
49       100            L6        AP good-BAS    2024  3142    NaN   
...      ...           ...                ...     ...   ...    ...   
22673    100            WN            Returns    2024  3142    NaN   
22674    100            WN            Returns    2024  3142    NaN   
22675    100            WN            Returns    2024  3142    NaN   
22676    100            WN            Returns    2024  3142    NaN   
22677    100            WN            Returns    2024  3142    NaN   
22678    100     

#creating the final file

In [64]:
loadfile_df = pd.DataFrame([])

In [65]:
#when the data is present both the system and not in sftp

In [66]:
doc_to_type = dict(zip(merged_df['Supplier number (Sales Line)'],
                           merged_df['Supplier number (MIAG)']))
doc_to_type

{nan: nan,
 '1000020870': '3500007956',
 '1000010074': '3500000073',
 '1000015545': '3500003274'}

Column populations

In [67]:
loadfile_df['COMPANY_CODE'] = merged_df['BUKRS']

# consider MIAG and if blank FI system/merged system

loadfile_df['SUPPLIER_NO']	= merged_df['Supplier number (Sales Line)'].where(merged_df['Supplier number (Sales Line)'].notna(), merged_df['suppl_no'])

loadfile_df['MIAG_SUPPLIER_NO'] = merged_df['Supplier number (MIAG)']

loadfile_df['MIAG_SUPPLIER_NO'] = loadfile_df['MIAG_SUPPLIER_NO'].fillna(loadfile_df['SUPPLIER_NO'].map(doc_to_type))

#loadfile_df['ORDER_NO'] = merged_df['AUFNR']

loadfile_df['DOC_TYPE'] = merged_df['Document type'].where(merged_df['Document type'].notna(), merged_df['Document_type'])

#sftp/fi/ic
loadfile_df['INVOICE_NO'] = merged_df['Invoice number'].where(merged_df['Invoice number'].notna(), merged_df['XBLNR'])

loadfile_df['INVOICE_DATE']	= merged_df['REDAT']

loadfile_df['DELIVERY_NOTE_NO'] = merged_df['LFSNR']

loadfile_df['TOTAL_AMT_DC']	= merged_df['GEBRF']

loadfile_df['TOTAL_VAT_DC']	= merged_df['GSMWF']

loadfile_df['CURRENCY']	= merged_df['WAERS_x']

condition = (loadfile_df['DOC_TYPE'] == 'MV')

loadfile_df['PRE_FINANCE_DATE'] = np.where(condition, merged_df['Value date'], '')

loadfile_df['GOODS_RECEIPT_NO']	= merged_df['WENUM']

loadfile_df['GOODS_RECEIPT_DATE'] = merged_df['WEDAT']

loadfile_df['INVOICE_ENTRY_DATE'] = merged_df['RGDAT'].where(merged_df['RGDAT'].notna(), merged_df['BLDAT'])

loadfile_df['INVOICE_STATUS'] = merged_df['INVOICE_STATUS']

loadfile_df['INVOICE_STATUS_INTERNAL'] = merged_df['ABGST'] 

loadfile_df['NET_DUE_DATE'] = merged_df['NET_DUE_DATE']

loadfile_df['DEBIT_NOTE_NO'] = merged_df['DEBNOTNO']

loadfile_df['REMITTANCE_ADVICE_NO'] = np.where(
    merged_df['INVOICE_STATUS'] == 'CLEARED', 
    merged_df['Remittance advice number'], 
    '')

loadfile_df['CLEARING_DATE'] =  merged_df['Value date'].where(merged_df['Value date'].notna(), merged_df['AUGDT'])

loadfile_df['DOCUMENT_NO']	= merged_df['BELNR'].where(merged_df['BELNR'].notna(), merged_df['Document number'])

loadfile_df['STORE_NO'] =  merged_df['store_or_dc']

loadfile_df['ARKTX'] =  merged_df['ARKTX_temp']

current_date = datetime.date.today()
formatted_current_date = current_date.strftime("%d.%m.%Y")
loadfile_df['MATCHING_DATE'] = formatted_current_date
loadfile_df['MATCH_STATUS'] = 'No Matching Requested'
loadfile_df['SYNC_DATE']	= formatted_current_date
loadfile_df['SYNC_STATUS'] = '1'
loadfile_df= loadfile_df.fillna('')

In [68]:
loadfile_df

COMPANY_CODE SUPPLIER_NO MIAG_SUPPLIER_NO DOC_TYPE INVOICE_NO  \
0             3142  1000010074       3500000073       L6    A933713   
1             3142  1000010074       3500000073       L6    A931806   
2             3142  1000010074       3500000073       L6    A931807   
3             3142  1000010074       3500000073       L6    A933715   
4             3142  1000010074       3500000073       L6    A933712   
5             3142  1000010074       3500000073       L6    A933714   
6             3142  1000010074       3500000073       L6    A935863   
7             3142  1000010074       3500000073       L6    A938030   
8             3142  1000010074       3500000073       L6    A940063   
9             3142  1000010074       3500000073       L6    A942302   
10            3142  1000010074       3500000073       L6    A944410   
11            3142  1000010074       3500000073       L6    A944411   
12            3142  1000010074       3500000073       L6    A947655   
13            3142  1000010074       3500000073       L6    A941080   
14            3142  1000010074       3500000073       L6    A943109   
15            3142  1000010074       3500000073       L6    A943108   
16            3142  1000015545       3500003274       L6    A931001   
17            3142  1000015545       3500003274       L6    A931003   
18            3142  1000015545       3500003274       L6    A932904   
19            3142  1000015545       3500003274       L6    A932905   
20            3142  1000015545       3500003274       L6    A935098   
21            3142  1000015545       3500003274       L6    A935099   
22            3142  1000015545       3500003274       L6    A936354   
23            3142  1000015545       3500003274       L6    A936987   
24            3142  1000015545       3500003274       L6    A936989   
25            3142  1000015545       3500003274       L6    A939320   
26            3142  1000015545       3500003274       L6    A939321   
27            3142  1000015545       3500003274       L6    A939322   
28            3142  1000015545       3500003274       L6    A941592   
29            3142  1000015545       3500003274       L6    A941593   
30            3142  1000015545       3500003274       L6    A941594   
31            3142  1000015545       3500003274       L6    A943566   
32            3142  1000015545       3500003274       L6    A943568   
33            3142  1000015545       3500003274       L6    A946095   
34            3142  1000015545       3500003274       L6    A946096   
35            3142  1000015545       3500003274       L6    A946097   
36            3142  1000020870       3500007956       L6    A932463   
37            3142  1000020870       3500007956       L6    A944195   
38            3142  1000020870       3500007956       L6    A930603   
39            3142  1000020870       3500007956       L6    A930602   
40            3142  1000020870       3500007956       L6    A932464   
41            3142  1000020870       3500007956       L6    A932462   
42            3142  1000020870       3500007956       L6    A934529   
43            3142  1000020870       3500007956       L6    A934528   
44            3142  1000020870       3500007956       L6    A936495   
45            3142  1000020870       3500007956       L6    A938817   
46            3142  1000020870       3500007956       L6    A938816   
47            3142  1000020870       3500007956       L6    A940935   
48            3142  1000020870       3500007956       L6    A940934   
49            3142  1000020870       3500007956       L6    A943001   
...            ...         ...              ...      ...        ...   
22673         3142  1000020870       3500007956       WN   24164270   
22674         3142  1000020870       3500007956       WN   24164556   
22675         3142  1000020870       3500007956       WN   24165312   
22676         3142  1000020870       3500007956       WN   24165780   
22677         3142  1000020870

In [69]:
current_datetime = datetime.datetime.now().strftime("%Y%m%d")
        
loadfile_df.to_csv(
            r"C:\Users\shivani.kumari\AppData\Local\Apps\projects\M360\processed_files\load.360.35." + current_datetime + ".001_test_internal.csv",
            sep=',', index=False)

loadfile_df.to_csv(
            r"C:\Users\shivani.kumari\AppData\Local\Apps\projects\M360\processed_files\load.360.35." + current_datetime + ".001_internal.csv",
            sep=';', index=False)